In [7]:
import numpy as np
import json
from astropy.table import Table, vstack, join
from astropy.io import fits
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '/global/homes/h/hrincon/python_tools')
import VoidVolume as vol
import VoidOverlap as vo
import VoidCatalog as vc
import VoidSlicePlots as vsp

from vast.voidfinder.postprocessing import mknum
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker
from vast.voidfinder.voidfinder import ra_dec_to_xyz
from vast.voidfinder.distance import z_to_comoving_dist


In [8]:
desi_gals_path = '/global/homes/h/hrincon/DESIVAST/galaxy_catalog/iron_ngc.fits'
sdss_gals_path = '/global/homes/h/hrincon/DESIVAST/alt_mag_cuts/SDSSK1/nsa_k1.fits'
masks_path = '/global/homes/h/hrincon/DESIVAST/galaxy_catalog/mask/alt_masks/masks.fits'

In [9]:
desi_gals = Table.read(desi_gals_path)
sdss_gals = Table.read(sdss_gals_path)
masks = fits.open(masks_path)

In [4]:
def select_mask(gals, mask_hdu, rmin, rmax):
    mask_checker = MaskChecker(0,
                            mask_hdu.data.astype(bool),
                            mask_hdu.header['MSKRES'],
                            rmin,
                            rmax)
    
    points_boolean = np.ones(len(gals), dtype = bool)
    
    if not np.all(np.isin(['x','y','z'], gals.colnames)):
        gals['Rgal']=z_to_comoving_dist(gals['redshift'].astype(np.float32),.315,1)
        tmp = ra_dec_to_xyz(gals)
        gals['x']=tmp[:,0]
        gals['y']=tmp[:,1]
        gals['z']=tmp[:,2]

    #Flag points that fall outside the mask
    for i in range(len(gals)):
        # The current point
        curr_pt = np.array([gals['x'][i],gals['y'][i],gals['z'][i]])
        # Declare if point is not in mask
        not_in_mask = mask_checker.not_in_mask(curr_pt)
        # Invert not_in_mask to tag points in the mask
        points_boolean[i] = not bool(not_in_mask)
    return gals[points_boolean]

In [31]:
desi_gals_masked = select_mask(desi_gals, masks['COMPFID'], 0, 332.38626)

In [35]:
sdss_gals_masked = select_mask(sdss_gals, masks['COMPFID'], 0, 332.38626)

In [36]:
# The DESI catalog has a -19.5 cut, while the SDSS catalog has no cut,
# if I remember right
len(desi_gals_masked), len(sdss_gals_masked)

(10285, 18085)

In [39]:
# Comparison of volume limited catalog
len(desi_gals_masked[desi_gals_masked['rabsmag']<-20]), len(sdss_gals_masked[sdss_gals_masked['rabsmag']<-20])


(6217, 7711)

In [42]:
#create a version with a larger redshift buffer for output
desi_gals_masked = select_mask(desi_gals, masks['COMPFID'], 0, 350)
sdss_gals_masked = select_mask(sdss_gals, masks['COMPFID'], 0, 350)

In [43]:
desi_gals_masked.write('desi_fiducial.fits')
sdss_gals_masked.write('sdss_fiducial.fits')

# Mocks

In [10]:
#Altmtl
for i in range(25):
    file = f'/global/homes/h/hrincon/DESIVAST/mocks/altmtl/altmtl{i}_ngc.fits'
    gals = Table.read(file)
    gals = select_mask(gals, masks['COMPFID'], 0, 332.38626)
    gals.write(f'./altmtl/altmtl{i}_ngc.fits')


In [14]:
#quaternary indexing
idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)
    
for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/mocks/HorizonRun4/DR7m_{i}.fits'
    gals = Table.read(file)
    gals['z'].name = 'redshift'
    gals = select_mask(gals, masks['COMPFID'], 0, 332.38626)
    gals.write(f'./HR4/DR7m_{i}.fits')